# 2. Dynamic Programming

Dynamic programming refers to methods which calculate the optimal policy given a perfect model of the environment as a Markov Decision process.
In real world applications a perfect model is usually not given and the amount of states yields great computational expense.


Dynamic programming is based on the bellman equation for the value function. <br>
**state value function** <br>
$v_\pi(s) = \sum_a \pi(s,a) \sum_{s'} P^a_{s s'}[R^a_{s s'} + \gamma v^\pi (s')] $

**action value function** <br>
$q_\pi(s) =  \sum_{s'} P^a_{s s'}[R^a_{s s'} + \gamma \sum_{a'}\pi(s',a')Q^\pi(s',a') ] $

This formula only says that the value of the current state $v_\pi(s)$ has to be equal to the value of all possible next states $s'$, weighted by the probability to get to that state + the reward to get on the way there. This probability depends on your policy (which action you choose in that state) $\pi(s,a)$ and the transition probability (how the environment reacts depending on your action) $P^a_{s s'}$.

### Value iteration
Starting with value iteration (right side of the image below), the bellman equation is applied to every state until the values of all states converge. After the value function for every state is known, the policy is extracted by acting greedy in every state/picking the next state with the highest value function.
The intuitive way to understand why it works, is that there has to be some goal state which the agent has to reach, for this state the value function already has the right value (e.g. +1 for a won game).
After applying the bellman equation the values around this goal state get shifted into the right direction. 
A few iterations later the values around the goal state get closer and closer to the correct value and those correct values get back-propagated to the next states, following this approach the right values back-propagate over the entire state space until they all converge.

![title](imgs/value_policy_iteration.png)

### Sample implementation for Value iteration

In [3]:
import gym
import numpy as np

env_name = 'FrozenLake-v0'
env = gym.make(env_name)
state_space = env.observation_space.n
action_space = env.action_space.n

v = np.zeros(state_space)
q = np.zeros([state_space, action_space])
gamma = 1.0
epsilon = 0.001

while(True):
    v_backup = v.copy()
    for s in range(state_space):
        for a in range(action_space):
            env_P = np.array(env.env.P[s][a])
            trans_prob = env_P[:,0]
            rewards = env_P[:,2]
            s_prime = env_P[:,1].astype(int)
            exp_r = np.dot(np.transpose(trans_prob), rewards)
            q[s,a] = exp_r +  gamma * np.dot(np.transpose(trans_prob), v[s_prime])
        v[s] = np.amax(q[s,:])
        
    if(np.sum(np.abs(v - v_backup)) < epsilon): # convergence criterium
        break

def render_value_fct_array(value_fct_array):
    len_x = int(np.sqrt(value_fct_array.shape[0]))
    array_2d = value_fct_array.reshape([ len_x, len_x])
    print(array_2d)

np.set_printoptions(linewidth = 150)
render_value_fct_array(v)
print(env.render())

[[0.82058182 0.81961477 0.81894265 0.81860067]
 [0.82087832 0.         0.52740189 0.        ]
 [0.82134266 0.82193808 0.76331139 0.        ]
 [0.         0.88124695 0.94061371 0.        ]]

SFFF
FHFH
FFFH
HFFG
None


The disadvantage of value iteration is that the right policy might have already been given before the values converge and the while loop could have been stopped earlier. This is why policy iteration which stops after the policy does not change after an iteration, is faster.
It took me a while to understand that, since the policy evaluation part is nearly the same as the entire value iteration, and it is repeated multiple times I thought it must be slower.
The key is that the values for each state get only initialized once so if the policy evaluation runs the second time, the policy has only changed a bit and it needs considerably less iterations to converge again which applies to all following repetitions of the loop.

### Sample implementation for policy iteration

In [9]:
# get the value function
import gym
import numpy as np

env_name = 'FrozenLake-v0'

env = gym.make(env_name)
state_space = env.observation_space.n
action_space = env.action_space.n

v = np.zeros(state_space)
q = np.zeros([state_space, action_space])
gamma = 0.99
epsilon = 1e-10
policy = np.random.randint(4, size=[state_space])

def policy_evaluation(env, policy, v):
    while(True):
        v_prev = v.copy()
        for s in range(state_space):
            temp = v.copy()
            a = policy[s]
            env_P = np.array(env.env.P[s][a])
            trans_prob = env_P[:,0]
            rewards = env_P[:,2]
            s_prime = env_P[:,1].astype(int)
            v[s] = np.sum( trans_prob * (rewards + gamma*v[s_prime]))
        if(np.sum(np.abs( v_prev - v )) < epsilon):
            return v
    
def policy_improvement(env, policy, v):
    for s in range(state_space):
        v_prime = np.zeros(action_space)
        for a in range(action_space):
            env_P = np.array(env.env.P[s][a])
            trans_prob = env_P[:,0]
            rewards = env_P[:,2]
            s_prime = env_P[:,1].astype(int)
            v_prime[a] = np.sum( trans_prob * (rewards + gamma*v[s_prime]) )
        policy[s] = np.argmax(v_prime)        
    
    return policy

def policy_iteration(env, policy, v):
    max_iter = 1000000
    for i in range(max_iter):
        old_policy = policy.copy()
        v = policy_evaluation(env, policy, v)
        policy = policy_improvement(env, policy, v)
        finished = np.all(old_policy == policy)
        
        if(finished):
            return policy, v

policy, v = policy_iteration(env, policy, v)
render_value_fct_array(v)
print(env.render())

[[0.54202593 0.49880319 0.47069569 0.4568517 ]
 [0.55845096 0.         0.35834807 0.        ]
 [0.59179874 0.64307982 0.61520756 0.        ]
 [0.         0.74172044 0.86283743 0.        ]]

SFFF
FHFH
FFFH
HFFG
None
